# Probability, Bayes' Theorem, (Conditional) Independence

In [1]:
import numpy as np
from helpers import print_table

# 1. Inference-by-Enumeration 

The Inference-by-Enumeration algorithm computes the answer to a probabilistic query $P(\mathbf{X} \mid \mathbf{E})$ exactly from the full joint distribution table (FJDT).

---
### 1.1. Implementation


<div class="alert alert-warning">
Implement the Inference-by-Enumeration algorithm. 
</div>

Implement the `inference_by_enumeration` function for a generic probabilistic query of the form $P(\mathbf{X} \mid \mathbf{E})$. Note that this version of the Inference-by-Enumeration algorithm computes the probabilistic query for all possible assignments to the evidence variables, not only for one specific assignment. The function must return one object:
- The answer to the probabilistic query, which is a `np.ndarray` with the same number of dimensions and the same variable order as the FJDT, but not the same size: The dimensions of non-query and non-evidence variables ($\mathbf{Z}$) must be converted to singleton dimensions, i.e., dimensions of size one.

For example, if we have a full joint distribution table of three binary variables (shape $2\times2\times2$) and we ask for the distributions of the first variable given the second variable, the result would be of shape $2\times2\times1$ (corresponding to two stacked conditional distribution tables).

In [2]:
def inference_by_enumeration(FJDT: np.ndarray, query_variable_indices: tuple, evidence_variable_indices: tuple=tuple()) -> np.ndarray:
    '''
    Computes the answer to a probabilistic query exactly from the full joint distribution table.
    :param table: The full joint distribution table as a np.ndarray.
    :param query_variable_indices: A tuple containing the indices of the query variables in the FJDT.
    :param evidence_variable_indices: A tuple containing the indices of the evidence variables in the FJDT.
    :returns: The answer to the probabilistic query; a `np.ndarray`.
    ''' 
    assert type(FJDT) == np.ndarray, "FJDT must be a np.ndarray"
    assert type(query_variable_indices) == tuple, "query_variable_indices must be a tuple"
    assert type(evidence_variable_indices) == tuple, "evidence_variable_indices must be a tuple"
    
    # compute the set of non-query and non-evidence variables, Z
    query_variables = query_variable_indices + evidence_variable_indices
    Z = tuple(set(range(FJDT.ndim)).difference(query_variables))
    #step 1: marginal distribution
    marg_dist = np.sum(FJDT,axis = Z,keepdims = True)

    #step 2: normalization constant
    norm_const = np.sum(marg_dist,axis = query_variable_indices,keepdims = True)
    #step 3: final result
    result = marg_dist/norm_const
    return result 

In [3]:
# create a full joint distribution table for three binary variables
ABC = np.ones((2,2,2)) / 8
# name the variable indices so we can refer to them more easily
A, B, C = 0, 1, 2
# check type & shape of result
assert type(inference_by_enumeration(ABC, (B, C), ())) == np.ndarray
# compute P(A)
assert inference_by_enumeration(ABC, (A,), ()).shape == (2, 1, 1)
# compute P(BC)
assert inference_by_enumeration(ABC, (B, C), ()).shape == (1, 2, 2)
# compute P(BC|A)
assert inference_by_enumeration(ABC, (B, C), (A,)).shape == (2, 2, 2)
# compute P(B|AC)
assert inference_by_enumeration(ABC, (B,), (C,A,)).shape == (2, 2, 2)

inference_by_enumeration(ABC, (B, C), (A,))

array([[[0.25, 0.25],
        [0.25, 0.25]],

       [[0.25, 0.25],
        [0.25, 0.25]]])

---
### 1.2. Full Joint Distribution Table

<br>
<center><img src="https://upload.wikimedia.org/wikipedia/commons/b/b9/Atlantic_blue_marlin.jpg" width="500" height="600">
<br>

Based on his experience, Santiago, an old Cuban fisherman, has learned that temperature and precipitation are the most prominent factors influencing marlin fishing. After decades of (more or less) successful years, he decides to retire and pass on his knowledge to a young apprentice. Since the apprentice received excellent grades in her probabilistic models class, he creates the following full joint distribution table $P(C, R, H)$ and hands it over to her:


<table style="border-collapse:collapse;border-spacing:0;width:500px"><tr><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center" rowspan="2">$P({C}, {R}, {H})$</th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$\neg r$<br></th><th style="font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top" colspan="2">$r$</th></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$\neg h$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">$h$</td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$\neg c$<br></td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.26</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.32</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.35</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.01<br></td></tr><tr><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center">$c$</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.04</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.01</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.004</td><td style="font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;text-align:center;vertical-align:top">0.006</td></tr></table>

In this table, $C$, $R$, and $H$ are the binary random variables encoding catch, rain, and hot, respectively. 
    

In [6]:
help(print_table)

Help on function print_table in module helpers:

print_table(probability_table: numpy.ndarray, variable_names: str) -> None
    Prints a probability distribution table.
    
    Parameters
    ----------
    probability_table : np.ndarray
        The probability distribution table
    variable_names : str
        A string containing the variable names, e.g., 'CDE'.
    
    Returns
    -------
    None



<div class="alert alert-warning">
Create a NumPy array that contains the full joint distribution table $P(C, R, H)$ as defined above. <b>Important</b>: Encode $C$, $R$, and $H$ in the first, second, and third dimension of the NumPy array, respectively. Use index 0 for event *False* and index 1 for event *True*. (1 point)
</div>

In [4]:
CRH = np.ones((2,2,2))
CRH[0,0,0] = 0.26
CRH[0,0,1] = 0.32
CRH[1,0,0] = 0.04
CRH[1,0,1] = 0.01
CRH[0,1,0] = 0.35 
CRH[0,1,1] = 0.01
CRH[1,1,0] = 0.004
CRH[1,1,1] = 0.006

Catch,Rain,Hot = 0,1,2
print_table(CRH, 'CRH')


,$r_0$,$r_0$,$r_1$,$r_1$
,$h_0$,$h_1$,$h_0$,$h_1$
$c_0$,0.260,0.320,0.350,0.010
$c_1$,0.040,0.010,0.004,0.006


In [5]:
assert CRH is not None, 'Store the result into the variable \'CRH\'!'
assert CRH.shape == (2,2,2), 'The full joint distribution table must have shape (2,2,2)'
assert CRH.sum() == 1, 'The probabilities of all atomic events must sum to one.'


---
### 1.3. Independence

<div class="alert alert-warning">
Show that catching the marlin is not independent of the weather being rainy, (i.e., $C \not \perp R$) by showing that the joint distribution of the variables is not equal to the product of the marginals.
</div>

In [6]:
CR = inference_by_enumeration(CRH, (Catch,Rain,), ())# store the joint distribution here
C_times_R = inference_by_enumeration(CRH, (Catch,), ()) * inference_by_enumeration(CRH, (Rain,), ()) # store the product of the marginal distributions here

CR = np.squeeze(CR,axis = 2)
C_times_R = np.squeeze(C_times_R,axis = 2)

print('Product of Marginals:')
print_table(C_times_R, 'CR')

print('Joint Probability:')
print_table(CR, 'CR')


Product of Marginals:


,$r_0$,$r_1$
$c_0$,0.592,0.348
$c_1$,0.038,0.022


Joint Probability:


,$r_0$,$r_1$
$c_0$,0.580,0.360
$c_1$,0.050,0.010


In [7]:
assert type(CR) == np.ndarray, 'Results must be NumPy arrays.'
assert type(C_times_R) == np.ndarray, 'Results must be NumPy arrays.'

assert CR.shape == (2,2), 'Results must be 2x2 arrays.'
assert C_times_R.shape == (2,2), 'Results must be 2x2 arrays.'